In [1]:
from pysparkling import *
from load_susy_into_df import susy_csv_to_df

In [2]:
path_to_csv = '/home/varsrao/Downloads/SUSY.csv'
chunksize = 100000
num_train_chunks = 10
num_test_chunks = 10

In [3]:
print('----Loading Dataset----')
ds_train_pd_df, ds_test_pd_df, target_col_name, target_col_idx, feature_col_names = susy_csv_to_df(
        path_to_csv, chunksize, num_train_chunks, num_test_chunks)
col_names = [target_col_name]+feature_col_names

----Loading Dataset----


In [4]:
print('----Creating H2O Context----')
hc = H2OContext.getOrCreate(spark)

----Creating H2O Context----
Connecting to H2O server at http://192.168.2.253:54325... successful.


H2O cluster uptime:,12 secs
H2O cluster timezone:,America/Toronto
H2O data parsing timezone:,UTC
H2O cluster version:,3.22.1.4
H2O cluster version age:,17 days
H2O cluster name:,sparkling-water-varsrao_local-1551835694879
H2O cluster total nodes:,1
H2O cluster free memory:,755 Mb
H2O cluster total cores:,4
H2O cluster allowed cores:,4
H2O cluster status:,"accepting new members, healthy"



Sparkling Water Context:
 * H2O name: sparkling-water-varsrao_local-1551835694879
 * cluster size: 1
 * list of used nodes:
  (executorId, host, port)
  ------------------------
  (driver,192.168.2.253,54325)
  ------------------------

  Open H2O Flow in browser: http://192.168.2.253:54325 (CMD + click in Mac OSX)

    


In [5]:
print('----Creating H2O Frame----')
import h2o
ds_f = h2o.H2OFrame(ds_train_pd_df, column_names=col_names)
ds_test_f = h2o.H2OFrame(ds_test_pd_df, column_names=col_names)
ds_test_f[target_col_name] = ds_test_f[target_col_name].asfactor()

h2o.cluster().timezone = "Etc/UTC"

----Creating H2O Frame----
Parse progress: |██████████████████████████████████████████████████████████████| 100%
Parse progress: |█████████████████████████████████████████████████████████| 100%


In [6]:
print('----Assembling Data----')
ds_f[target_col_name] = ds_f[target_col_name].asfactor()
predictor_columns = ds_f.drop(target_col_name).col_names
response_column = target_col_name

----Assembling Data----


In [7]:
print('----Training Model----')
from h2o.transforms.decomposition import H2OPCA
pca_decomp = H2OPCA(k=10, transform="NONE", pca_method="Power", impute_missing=True)
pca_decomp.train(x=predictor_columns, training_frame=ds_f)
pca_decomp.summary()

----Training Model----
pca Model Build progress: |███████████████████████████████████████████████| 100%
Importance of components: 


,pc1,pc2,pc3,pc4,pc5,pc6,pc7,pc8,pc9,pc10
Standard deviation,3.6184818,1.3119014,1.1886384,1.1455862,1.0530202,1.0179253,0.8130579,0.7739041,0.6454309,0.5923058
Proportion of Variance,0.5888008,0.0773959,0.0635353,0.0590162,0.0498642,0.0465959,0.0297275,0.0269333,0.0187333,0.0157764
Cumulative Proportion,0.5888008,0.6661967,0.7297320,0.7887482,0.8386124,0.8852082,0.9149358,0.9418691,0.9606024,0.9763788


In [8]:
print('----Testing Model----')
predictions = pca_decomp.predict(ds_test_f)
print(predictions.ncols)

print('----End----')

----Testing Model----
pca prediction progress: |████████████████████████████████████████████████| 100%
10
----End----
